In [5]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from thesis_toolbox.plot.plot_climatology import plot_climatology_500hpa
from thesis_toolbox.plot.tools import map_large_scale, add_letter, add_colorbar
from thesis_toolbox.process_era5 import resample_seasonal
import numpy as np
from thesis_toolbox.composites.create_composites import detrend_timeseries,select_years_to_composite
from thesis_toolbox.composites.setup_thesis_data import geopot_wind_composite
from thesis_toolbox.plot.plot_composites import plot_500hPa_composite, plot_which_years_composited
from thesis_toolbox.utils import get_locations_CLP,read_receptor_composite
import glob
import matplotlib as mpl
%matplotlib inline

In [6]:
if 'snakemake' in globals():
    to_file = True
    path='./'
    kind = snakemake.wildcards.kind
else:
    path = '../../../'
    to_file= False
    kind = 'total_deposition'

In [10]:
def plot_composite(composite,axes, locs, scale=0.5, u=2, add_label=False):
    for ax ,loc_name in zip(axes.ravel(),locs.index):
        ax.tick_params(axis="y", which='both',right=True, left=True, labelleft=False, labelright=False)
        ax.tick_params(axis="x", which='both',bottom=True, top=False, labelbottom=False)
        plot_500hPa_composite(composite, ax=ax,  x_qk=0.81, y_qk=1.12, receptor_loc=locs.loc[loc_name,:].to_list(), 
                               receptor_name=loc_name
                          , vector_scale=scale,angles='uv', colorbar=False, hatches='xxx', hatch_color='dimgrey',
                                  xticks=[20,60,100,140,180])
        if add_label:
            if loc_name=='BADOE':
                loc_name='BAODE'
            ax.set_ylabel(loc_name)

## Spring and Winter depo composites 500hPa

In [8]:

locs_df = get_locations_CLP().drop('YINCHUAN')

data_path =path + 'downloads/intermediate_results/'

DJF_20mu_composite = read_receptor_composite(locs_df.index, 
                                             path, 'windspeed_geopot_500hPa', 
                                             '20micron','DJF', std='4_rank', kind=kind)


DJF_2mu_composite = read_receptor_composite(locs_df.index, path, 'windspeed_geopot_500hPa', 
                                            '2micron','DJF', std='4_rank', kind=kind)


MAM_20mu_composite = read_receptor_composite(locs_df.index, path, 'windspeed_geopot_500hPa', 
                                             '20micron','MAM', std='4_rank', kind=kind)
MAM_2mu_composite = read_receptor_composite(locs_df.index, path, 'windspeed_geopot_500hPa', 
                                            '2micron','MAM', std='4_rank', kind=kind)

In [1]:
fig,ax = plt.subplots(nrows=6, ncols=4,  figsize=(8.3*1.7, 11.7*1.2), subplot_kw={'projection':ccrs.PlateCarree()})


fig.suptitle('Clay size-bin                                                           Silt size-bin',x=0.515, y=0.91,
             ha='center', va='center', fontsize=16)


plot_composite(DJF_2mu_composite, ax[:,0], locs_df, add_label=True)
plot_composite(MAM_2mu_composite,ax[:,1], locs_df)
plot_composite(DJF_20mu_composite,ax[:,2], locs_df)
plot_composite(MAM_20mu_composite,ax[:,-1], locs_df)
for axi,l in zip(ax[:,0].ravel(),locs_df.index):
    axi.tick_params(axis="y", which='both',right=True, left=True, labelleft=True, labelright=False)    

for axi in ax[-1,:].ravel():
#     axi.set_xticks([20, 60, 100, 140, 180])
#     axi.xaxis.set_ticklabels(['20°E', '60°E', '100°E', '140°E','180°E'])
    axi.tick_params(axis="x", which='both',bottom=True, top=False, labelbottom=True)
ax[0,0].set_title('DJF')
ax[0,1].set_title('MAM')
ax[0,2].set_title('DJF')
ax[0,3].set_title('MAM')
fig.subplots_adjust(hspace=0.3, wspace=0.13)
cax = fig.add_axes([0.33,0.06,0.365, 0.02])
cmap = mpl.cm.bwr
add_letter(ax.T, y=1.1, fontsize=10)
fig.colorbar(mpl.cm.ScalarMappable(mpl.colors.BoundaryNorm(np.linspace(-7.5,7.5,16),cmap.N, extend='both'), 
                                          cmap=cmap),cax=cax, extend='both', orientation='horizontal', 
            label='500 hPa wind speed composite anomalies [m/s]')

if to_file:
    plt.savefig(snakemake.output.composite_facet_plot, bbox_inches='tight', dpi=300)

NameError: name 'plt' is not defined